In [3]:
import pandas as pd

df = pd.read_csv("Bootcamp_Santander_2025.csv")
user_ids = df["name"].tolist()
print(user_ids)




['José Ferreira', 'Ana Silva', 'Bruno Oliveira', 'Carla Souza', 'Diego Santos']


In [9]:
!pip install -U google-genai

Erro ao processar José Ferreira: 404 models/gemini-1.5-flash is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.
Erro ao processar Ana Silva: 404 models/gemini-1.5-flash is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.
Erro ao processar Bruno Oliveira: 404 models/gemini-1.5-flash is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.
Erro ao processar Carla Souza: 404 models/gemini-1.5-flash is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.
Erro ao processar Diego Santos: 404 models/gemini-1.5-flash is not found for API version v1beta, or is not supported for gene

In [4]:
from google import genai
from google.genai import types
import time

API_KEY = 'CHAVE_API'

INPUT_FILE = r"Bootcamp_Santander_2025.csv"
OUTPUT_FILE = r"Bootcamp_Santander_2025_enriched.csv"

def get_gemini_client():
    if API_KEY == 'CHAVE_API' or not API_KEY:
        raise ValueError("Erro de Configuração: Você não usou uma chave de API Key")
    
    return genai.Client(api_key=API_KEY)

def generate_marketing_message(client, user_row):
   
    name = user_row['name']
    balance = float(user_row['account_balance'])
    
    if balance > 2000:
        focus = "investimentos exclusivos e diversificação de carteira"
    elif balance > 500:
        focus = "poupança e objetivos de curto prazo"
    else:
        focus = "organização financeira e controle de gastos"

    prompt = (
        f"Você é um especialista em marketing bancário."
        f"Escreva uma notificação curta (máximo 1 frase) para o cliente {name}. "
        f"Foco: {focus}. "
        "Tom: Profissional mas amigável. Não mencione valores numéricos específicos."
    )

    try:
        response = client.models.generate_content(
            model="gemini-1.5-flash",
            contents=prompt,
            config=types.GenerateContentConfig(
                temperature=0.7,
                max_output_tokens=100
            )
        )
        return response.text.strip()
    except Exception as e:
        print(f"Aviso: Falha na IA para {name}. Confira as novidades exclusivas no seu App Santander!")
        return "Confira as novidades exclusivas no seu App Santander!"

def run_etl():
    print("Iniciando Pipeline ETL...")
    
    # 1. EXTRAÇÃO
    print(f"Lendo dados de: {INPUT_FILE}")
    try:
        df = pd.read_csv(INPUT_FILE)
        print(f"   -> {len(df)} registros carregados.")
    except FileNotFoundError:
        print(f"Erro: Arquivo não encontrado em {INPUT_FILE}")
        return

    try:
        client = get_gemini_client()
    except ValueError as e:
        print(e)
        return

    # 2. TRANSFORMAÇÃO
    print("Gerando mensagens personalizadas (IA)...")
    marketing_messages = []
    
    for index, row in df.iterrows():
        print(f"   [{index+1}/{len(df)}] Processando: {row['name']}")
        
        msg = generate_marketing_message(client, row)
        marketing_messages.append(msg)
        
        time.sleep(1.5)

    df['news_notification'] = marketing_messages

    # 3. CARREGAMENTO
    print(f"Salvando resultados em: {OUTPUT_FILE}")
    try:
        df.to_csv(OUTPUT_FILE, index=False)
        print("Pipeline finalizado com sucesso!")
    except PermissionError:
        print("Erro: Não foi possível salvar. Verifique se o arquivo já está aberto.")

if __name__ == "__main__":
    run_etl()

Iniciando Pipeline ETL...
Lendo dados de: Bootcamp_Santander_2025.csv
   -> 5 registros carregados.
Erro de Configuração: Você não usou uma chave de API Key
